<a href="https://colab.research.google.com/github/HuinaGabriela/Image-Based-Product-Recommender/blob/main/DatasetEPr%C3%A9Processamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dataset e Pré-processamento):
#   Filtra o DataFrame df para 4 categorias escolhidas
#   Seleciona 200 imagens por categoria
#   Copia as imagens para data/filtrado_200/<categoria>/
#   cria o drive e salva o arquivo CSV

In [3]:
# CUIDADO: Isso apaga todos os kaggle*.json, inclusive o principal
# import os
# import glob

# arquivos = glob.glob("kaggle*.json")

# for arquivo in arquivos:
#     os.remove(arquivo)

# print("Todos os arquivos kaggle*.json foram removidos.")



In [ ]:
# Bibliotecas
from google.colab import files
import os

import shutil
import pandas as pd
from tqdm.notebook import tqdm


In [5]:
# Faça uma conta no https://www.kaggle.com
# Procure por "API", "criar um novo tokem" (Um arquivo chamado kaggle.json será automaticamente baixado)

print("Faça o upload do seu arquivo `kaggle.json`. Ele não será exibido.")

# Faz o upload, mas não mostra o conteúdo (não imprime a variável)
_ = files.upload()

# Verifica se o arquivo foi realmente enviado
if not os.path.exists("kaggle.json"):
    raise FileNotFoundError("Erro: você precisa enviar o arquivo 'kaggle.json'.")



Faça o upload do seu arquivo `kaggle.json`. Ele não será exibido.


Saving kaggle.json to kaggle.json


In [6]:
# Move para a pasta padrão da API do Kaggle e protege o acesso
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print(" Upload realizado com sucesso e chave configurada com segurança.")


 Upload realizado com sucesso e chave configurada com segurança.


In [7]:
# Baixar o dataset do Kaggle - Fashion Product Images (Small)

# Instala o pacote do Kaggle (se ainda não instalou)
!pip install -q kaggle

# Baixa o dataset
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

# Extrai o conteúdo na pasta "data"
!unzip -q fashion-product-images-small.zip -d data

print(" Dataset baixado e extraído com sucesso.")



Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small
License(s): MIT
100% 565M/565M [00:06<00:00, 165MB/s]
100% 565M/565M [00:06<00:00, 95.8MB/s]
 Dataset baixado e extraído com sucesso.


In [8]:
# Lista todas as pastas do diretório atual
pastas = [p for p in os.listdir() if os.path.isdir(p)]
print(" Pastas no diretório atual:")
for p in pastas:
    print("•", p)

 Pastas no diretório atual:
• .config
• data
• sample_data


In [9]:
# Visualizar e contar as categorias disponíveis
import pandas as pd

# Lê o CSV (com tratamento de erros)
df = pd.read_csv("data/styles.csv", on_bad_lines='skip')
df.dropna(inplace=True)

print(" CSV carregado. Total de linhas:", len(df))
df.head()


 CSV carregado. Total de linhas: 44077


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [10]:
# Ver as subcategorias e contagens
subcats = df['subCategory'].value_counts()
print(" Subcategorias disponíveis:\n")
print(subcats)


 Subcategorias disponíveis:

subCategory
Topwear                     15387
Shoes                        7323
Bags                         3053
Bottomwear                   2686
Watches                      2542
Innerwear                    1806
Jewellery                    1079
Eyewear                      1073
Fragrance                    1001
Sandal                        961
Wallets                       925
Flip Flops                    913
Belts                         811
Socks                         698
Dress                         478
Loungewear and Nightwear      464
Saree                         427
Lips                          425
Headwear                      293
Nails                         278
Makeup                        263
Ties                          258
Accessories                   129
Scarves                       118
Cufflinks                     108
Apparel Set                   106
Free Gifts                    104
Stoles                         90
Skin   

In [16]:
# Caminhos
caminho_imagens = "data/images"
destino_base = "data/filtrado_200"

# Categorias que você quer usar
categorias_escolhidas = ['Bags', 'Eyewear', 'Jewellery', 'Watches']

# Filtrar até 200 imagens por categoria
df_amostrado = pd.DataFrame()

for cat in categorias_escolhidas:
    df_cat = df[df['subCategory'] == cat].sample(n=200, random_state=42)
    df_amostrado = pd.concat([df_amostrado, df_cat], ignore_index=True)

print(f"Total de imagens selecionadas: {len(df_amostrado)} (200 por categoria)")

# Salvar CSV com os metadados filtrados
csv_path = os.path.join(destino_base, "metadata_filtrado.csv")
os.makedirs(destino_base, exist_ok=True)  # Garante que a pasta exista antes de salvar o CSV
df_amostrado.to_csv(csv_path, index=False)
print(f" CSV salvo com sucesso em: {csv_path}")

# Criar pastas por categoria
for categoria in categorias_escolhidas:
    pasta_destino = os.path.join(destino_base, categoria)
    os.makedirs(pasta_destino, exist_ok=True)

# Copiar as imagens para suas respectivas pastas
erros = []
for _, row in tqdm(df_amostrado.iterrows(), total=len(df_amostrado), desc="Copiando imagens"):
    nome_arquivo = f"{row['id']}.jpg"
    categoria = row['subCategory']

    origem = os.path.join(caminho_imagens, nome_arquivo)
    destino = os.path.join(destino_base, categoria, nome_arquivo)

    if os.path.exists(origem):
        shutil.copy(origem, destino)
    else:
        erros.append(nome_arquivo)

print(" Cópia concluída!")
print(f" Imagens não encontradas: {len(erros)}")



Total de imagens selecionadas: 800 (200 por categoria)
 CSV salvo com sucesso em: data/filtrado_200/metadata_filtrado.csv


Copiando imagens:   0%|          | 0/800 [00:00<?, ?it/s]

 Cópia concluída!
 Imagens não encontradas: 0


In [17]:
import os

# Caminho base onde o CSV deveria estar
destino_base = "data/filtrado_200"
caminho_csv = os.path.join(destino_base, "metadata_filtrado.csv")

# Verifica se o arquivo existe
if os.path.exists(caminho_csv):
    print(f"CSV encontrado em: {caminho_csv}")
else:
    print(f"CSV NÃO encontrado em: {caminho_csv}")
    print("Verifique se o caminho está correto ou se o arquivo foi gerado.")


✅ CSV encontrado em: data/filtrado_200/metadata_filtrado.csv


In [23]:
from google.colab import drive
drive.mount('/content/drive')

# Cria pasta no Drive e move os dados
!mkdir -p /content/drive/MyDrive/meu_projeto_recomendacao
!cp -r data/filtrado_200 /content/drive/MyDrive/meu_projeto_recomendacao/


Mounted at /content/drive


In [26]:
!ls /content/drive/MyDrive/meu_projeto_recomendacao/filtrado_200


Bags  Eyewear  Jewellery  metadata_filtrado.csv  Watches
